In [20]:
import os
import numpy as np
import pandas as pd
import time
from utm import *
from tqdm import tqdm, tqdm_pandas
from osgeo import osr
import coordTransform


In [21]:
date = '20161103' #TODO
day_begin = '07:30:00' #TODO
day_end = '10:30:00' #TODO
day_begin_name = 730 #TODO
day_end_name = 1030 #TODO
DATE = 3  #TODO
Date = '03' #TODO

ORDER_FILE_PATH_WIN = f'F:/大学/第40期PRP/特征提取/坐标转换后的数据/201611{Date}-{day_begin_name}-{day_end_name}.csv' 
DST_PATH = f'F:/大学/第40期PRP/特征提取/Kepler可视化文件/{date}--{day_begin_name}-{day_end_name}.csv' 
TIME_INTERVAL = 20
SPACE_INTERVAL = 70

In [22]:
df = pd.read_csv(ORDER_FILE_PATH_WIN)

In [23]:
time1 = f'{date} {day_begin}'
time2 = f'{date} {day_end}'
stamp1 = time.mktime(time.strptime(time1, '%Y%m%d %H:%M:%S'))

# 时间窗划分
df['time_id'] = df.timestamp.apply(lambda x: (x - stamp1)//TIME_INTERVAL)

# 空间网格划分
# 1.计算左边界和上边界，左右-x， 上下-y
left = df['x'].min()
down = df['y'].max()

# 2.生成横向和纵向索引
df['row_id'] = df['y'].apply(lambda y: (y - down)//SPACE_INTERVAL)
df['col_id'] = df['x'].apply(lambda x: (x - left)//SPACE_INTERVAL)

print (f'{DATE} 已生成时空索引')

df = df.dropna()
print (f'{DATE} 已处理空值, 当前数据条数：',len(df))

3 已生成时空索引
3 已处理空值, 当前数据条数： 850352


In [24]:
df

,driver_ID,order_ID,timestamp,lon,lat,x,y,time_id,row_id,col_id
0,hdBvycdlh2yiwacrxxlhqqva9a6.tqzg,tmBmn9g9abvmuj7oAvthuoulgk47yrpn,1478139281,104.040276,30.693340,1.158171e+07,3.592987e+06,494.0,-69.0,0.0
1,hdBvycdlh2yiwacrxxlhqqva9a6.tqzg,tmBmn9g9abvmuj7oAvthuoulgk47yrpn,1478139284,104.040615,30.693230,1.158175e+07,3.592973e+06,494.0,-69.0,1.0
2,hdBvycdlh2yiwacrxxlhqqva9a6.tqzg,tmBmn9g9abvmuj7oAvthuoulgk47yrpn,1478139287,104.041004,30.693099,1.158179e+07,3.592956e+06,494.0,-69.0,2.0
3,hdBvycdlh2yiwacrxxlhqqva9a6.tqzg,tmBmn9g9abvmuj7oAvthuoulgk47yrpn,1478139290,104.041373,30.692988,1.158183e+07,3.592942e+06,494.0,-69.0,2.0
4,hdBvycdlh2yiwacrxxlhqqva9a6.tqzg,tmBmn9g9abvmuj7oAvthuoulgk47yrpn,1478139293,104.041722,30.692867,1.158187e+07,3.592926e+06,494.0,-70.0,3.0
...,...,...,...,...,...,...,...,...,...,...
850347,odqxr9cei1vkro6psmkdvnokjnfawiod,ioAmulkag5xowp3rzBopwrtf9lb-stwe,1478134573,104.042032,30.666638,1.158191e+07,3.589531e+06,258.0,-118.0,3.0
850348,odqxr9cei1vkro6psmkdvnokjnfawiod,ioAmulkag5xowp3rzBopwrtf9lb-stwe,1478134576,104.041584,30.666869,1.158186e+07,3.589561e+06,258.0,-118.0,2.0
850349,odqxr9cei1vkro6psmkdvnokjnfawiod,ioAmulkag5xowp3rzBopwrtf9lb-stwe,1478134579,104.041145,30.667090,1.158181e+07,3.589590e+06,258.0,-117.0,2.0
850350,odqxr9cei1vkro6psmkdvnokjnfawiod,ioAmulkag5xowp3rzBopwrtf9lb-stwe,1478134582,104.040716,30.667301,1.158176e+07,3.589617e+06,259.0,-117.0,1.0


In [38]:

# 下面开始时空特征提取

#1. 计算瞬时速度
# 排序：先按司机排，同司机按订单排，同订单再按时间排
print('正在计算瞬时速度')
df = df.sort_values(by = ['driver_ID', 'order_ID', 'timestamp']).reset_index(drop = True)
# 将订单id下移一行，用于判断前后数据是否属于同一订单
df['orderFlag'] = df['order_ID'].shift(1)
df['identi'] = (df['orderFlag'] == df['order_ID']) #一个由boolean构成的列，方便后面所有shift完成了之后再删除分界行
# 将坐标，时间戳下移一行，匹配相应轨迹点
df['x1'] = df['x'].shift(1)
df['y1'] = df['y'].shift(1)
df['timestamp1'] = df['timestamp'].shift(1)
# 将不属于同一订单的轨迹点删除
df = df[df['identi'] == True]
# 计算相邻轨迹点之间的距离和相差时间
# 距离采用欧式距离
dist = np.sqrt(np.square(df['x'].values - df['x1'].values) + np.square(df['y'].values - df['y1'].values))
ttime = df['timestamp'].values - df['timestamp1'].values
# 计算速度
df['speed'] = dist/ttime
# 删除临时数据
df = df.drop(columns = ['x1', 'y1', 'orderFlag', 'timestamp1', 'identi'])
print('已生成速度')

# 2.计算瞬时加速度
print('正在计算加速度')
df['speed1'] = df['speed'].shift(1)
df['timestamp1'] = df['timestamp'].shift(1)
df['identi'] = df['order_ID'].shift(1)
df = df[df.identi == df.order_ID]
df['acc'] = (df.speed - df.speed1)/(df.timestamp - df.timestamp1)
df = df.drop(columns = ['speed1', 'timestamp1', 'identi'])
print('已生成加速度')

df = df.reset_index(drop = True)

正在计算瞬时速度
已生成速度
正在计算加速度
已生成加速度


In [39]:
df

,driver_ID,order_ID,timestamp,lon,lat,x,y,time_id,row_id,col_id,speed,acc
0,gaAnkakmi9Cjpa-jqtnnuvmigmi1ssrm,iezqtjlc40AoAa7mppheyrlhbb85zmnh,1478133093,104.042728,30.712955,1.158198e+07,3.595527e+06,184.0,-32.0,4.0,20.460028,0.613079
1,gaAnkakmi9Cjpa-jqtnnuvmigmi1ssrm,iezqtjlc40AoAa7mppheyrlhbb85zmnh,1478133096,104.043257,30.713024,1.158204e+07,3.595536e+06,184.0,-32.0,5.0,19.835345,-0.208228
2,gaAnkakmi9Cjpa-jqtnnuvmigmi1ssrm,iezqtjlc40AoAa7mppheyrlhbb85zmnh,1478133099,104.043765,30.713073,1.158210e+07,3.595542e+06,184.0,-32.0,6.0,18.989645,-0.281900
3,gaAnkakmi9Cjpa-jqtnnuvmigmi1ssrm,iezqtjlc40AoAa7mppheyrlhbb85zmnh,1478133102,104.044095,30.713112,1.158214e+07,3.595547e+06,185.0,-32.0,6.0,12.328619,-2.220342
4,gaAnkakmi9Cjpa-jqtnnuvmigmi1ssrm,iezqtjlc40AoAa7mppheyrlhbb85zmnh,1478133108,104.044912,30.713201,1.158223e+07,3.595559e+06,185.0,-32.0,8.0,15.291237,0.493770
...,...,...,...,...,...,...,...,...,...,...,...,...
824826,vpzzwcahd4EpBd-corujtnkhakh@BpAm,jlvzmkeoh3Bxpk3nxvsewox88ab8Cqrk,1478132766,104.112291,30.660194,1.158973e+07,3.588697e+06,168.0,-130.0,115.0,24.149225,-1.390223
824827,vpzzwcahd4EpBd-corujtnkhakh@BpAm,jlvzmkeoh3Bxpk3nxvsewox88ab8Cqrk,1478132771,104.112291,30.659664,1.158973e+07,3.588628e+06,168.0,-131.0,115.0,13.711458,-2.087553
824828,vpzzwcahd4EpBd-corujtnkhakh@BpAm,jlvzmkeoh3Bxpk3nxvsewox88ab8Cqrk,1478132778,104.112251,30.658075,1.158972e+07,3.588423e+06,168.0,-134.0,115.0,29.388777,2.239617
824829,vpzzwcahd4EpBd-corujtnkhakh@BpAm,jlvzmkeoh3Bxpk3nxvsewox88ab8Cqrk,1478132781,104.112251,30.657555,1.158972e+07,3.588356e+06,169.0,-135.0,115.0,22.420803,-2.322658


In [40]:
# 下面计算集体/网格平均特征

# 1. 网格平均速度：先求每辆车在网格中的平均速度，然后求网格中所有个体平均速度的军制
# 基于时空网格和估计id分组
print('正在生成平均网格速度')
orderGrouped = df.groupby(['lon', 'lat' ,'row_id', 'col_id', 'time_id', 'driver_ID'])
# 网格在每个时刻（时间窗）的平均速度
grouped_speed = orderGrouped.speed.mean().reset_index()
grouped_speed = grouped_speed.groupby(['lon', 'lat' ,'row_id', 'col_id', 'time_id', 'driver_ID'])
grid_speed = grouped_speed.speed.mean()
# 去除异常值
grid_speed = grid_speed.clip(grid_speed.quantile(0.05), grid_speed.quantile(0.95))
print('已生成网格平均速度')

# 2. 网格平均加速度
print('正在生成网格平均加速度')
gridGrouped = df.groupby(['lon', 'lat' ,'row_id', 'col_id', 'time_id', 'driver_ID'])
grid_acc = gridGrouped.acc.mean()
print('已生成网格平均加速度')


正在生成平均网格速度
已生成网格平均速度
正在生成网格平均加速度
已生成网格平均加速度


In [41]:
grid_speed

lon         lat        row_id  col_id  time_id  driver_ID                       
104.039729  30.657688  -135.0  0.0     454.0    scAorjjag5zjBo_iyuvcplrf7lfaFqom    11.469894
104.039739  30.657688  -135.0  0.0     461.0    ghoqucicg.Dusa1oyrnkCps8bkh5ttyk    11.839897
104.039749  30.660996  -128.0  0.0     219.0    ugBulkik71Fqyg3ouokgCvjbibd-uuuc    12.040412
                                       222.0    jdwnoglcc7Auta0myptcCznnmba3xksm    18.387601
                                       347.0    ultsjgma89voyi4msypfqvxj8l8@Bmpc    11.237784
                                                                                      ...    
104.127004  30.661507  -127.0  138.0   269.0    naByr9bf76xqubbqpxqfBAqehdc_BxBl    17.632109
104.127014  30.661517  -127.0  138.0   468.0    inwtw9dfi_Ftvi_owwtlqnw87af4uwsc    19.022417
            30.661507  -127.0  138.0   343.0    njxusmfa62xxwh9erpmdpnjckla9tqwo    15.914934
104.127034  30.661527  -127.0  138.0   313.0    lapztginj3ytzibrDtifCprdd

In [42]:
grid_acc

lon         lat        row_id  col_id  time_id  driver_ID                       
104.039729  30.657688  -135.0  0.0     454.0    scAorjjag5zjBo_iyuvcplrf7lfaFqom    1.339157
104.039739  30.657688  -135.0  0.0     461.0    ghoqucicg.Dusa1oyrnkCps8bkh5ttyk    0.078753
104.039749  30.660996  -128.0  0.0     219.0    ugBulkik71Fqyg3ouokgCvjbibd-uuuc    0.023581
                                       222.0    jdwnoglcc7Auta0myptcCznnmba3xksm   -0.054544
                                       347.0    ultsjgma89voyi4msypfqvxj8l8@Bmpc    0.850043
                                                                                      ...   
104.127004  30.661507  -127.0  138.0   269.0    naByr9bf76xqubbqpxqfBAqehdc_BxBl   -1.187726
104.127014  30.661517  -127.0  138.0   468.0    inwtw9dfi_Ftvi_owwtlqnw87af4uwsc    0.449095
            30.661507  -127.0  138.0   343.0    njxusmfa62xxwh9erpmdpnjckla9tqwo    0.286259
104.127034  30.661527  -127.0  138.0   313.0    lapztginj3ytzibrDtifCprddmc-sqwd  

In [43]:
# 3.网格浮动车流量
print('正在生成网格浮动车数量')
grouped_volume = orderGrouped.speed.last().reset_index() #每个时空网格中的每个order只保留一辆（用last()来取）
grouped_volume = grouped_volume.groupby(['lon', 'lat' ,'row_id', 'col_id', 'time_id', 'driver_ID'])
grid_volume = grouped_volume['speed'].size()
grid_volume = grid_volume.clip(grid_volume.quantile(0.05), grid_volume.quantile(0.95))
print('已生成网格浮动车数量')
grid_volume.unique()

正在生成网格浮动车数量
已生成网格浮动车数量


array([1], dtype=int64)

In [44]:
# 4.网格车速标准差
print('正在生成网格车速标准差')
grid_v_std = gridGrouped.speed.std(ddof=0)
# 去除异常值
grid_v_std = grid_v_std.clip(grid_v_std.quantile(0.05), grid_v_std.quantile(0.95))
print('已生成网格车速标准差')
grid_v_std.unique()

正在生成网格车速标准差
已生成网格车速标准差


array([0.])

In [45]:



# 5.网格平均停车次数
print('正在生成网格平均停车次数')
stopNum = gridGrouped.speed.agg(lambda x: (x==0).sum())
grid_stop = pd.concat((stopNum, grid_volume), axis = 1)
grid_stop['stopNum'] = stopNum.values/ grid_volume.values
grid_stop = grid_stop['stopNum']
grid_stop = grid_stop.clip(0, grid_stop.quantile(0.95))
print('已生成网格平均停车次数')
grid_stop.unique()


正在生成网格平均停车次数
已生成网格平均停车次数


array([0., 1., 2.])

In [46]:
feature = pd.concat([grid_speed, grid_acc, grid_volume, grid_v_std, grid_stop], axis = 1).reset_index()

In [47]:
feature

,lon,lat,row_id,col_id,time_id,driver_ID,speed,acc,speed,speed,stopNum
0,104.039729,30.657688,-135.0,0.0,454.0,scAorjjag5zjBo_iyuvcplrf7lfaFqom,11.469894,1.339157,1,0.0,0.0
1,104.039739,30.657688,-135.0,0.0,461.0,ghoqucicg.Dusa1oyrnkCps8bkh5ttyk,11.839897,0.078753,1,0.0,0.0
2,104.039749,30.660996,-128.0,0.0,219.0,ugBulkik71Fqyg3ouokgCvjbibd-uuuc,12.040412,0.023581,1,0.0,0.0
3,104.039749,30.660996,-128.0,0.0,222.0,jdwnoglcc7Auta0myptcCznnmba3xksm,18.387601,-0.054544,1,0.0,0.0
4,104.039749,30.660996,-128.0,0.0,347.0,ultsjgma89voyi4msypfqvxj8l8@Bmpc,11.237784,0.850043,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
684327,104.127004,30.661507,-127.0,138.0,269.0,naByr9bf76xqubbqpxqfBAqehdc_BxBl,17.632109,-1.187726,1,0.0,0.0
684328,104.127014,30.661517,-127.0,138.0,468.0,inwtw9dfi_Ftvi_owwtlqnw87af4uwsc,19.022417,0.449095,1,0.0,0.0
684329,104.127014,30.661507,-127.0,138.0,343.0,njxusmfa62xxwh9erpmdpnjckla9tqwo,15.914934,0.286259,1,0.0,0.0
684330,104.127034,30.661527,-127.0,138.0,313.0,lapztginj3ytzibrDtifCprddmc-sqwd,17.304050,-0.017318,1,0.0,0.0


In [48]:
# feature.columns = ['lon', 'lat' ,'row_id','col_id', 'time_id', 'order_ID' 'aveSpeed', 'gridAcc', 'volume', 'speedStd', 'stopNum']
# print('已整理网格特征')
# feature.sort_values(['stopNum']).reset_index(drop=True)
# feature['date'] = DATE

In [49]:
# feature.to_csv(DST_PATH, index=False)

In [50]:
feature

,lon,lat,row_id,col_id,time_id,driver_ID,speed,acc,speed,speed,stopNum
0,104.039729,30.657688,-135.0,0.0,454.0,scAorjjag5zjBo_iyuvcplrf7lfaFqom,11.469894,1.339157,1,0.0,0.0
1,104.039739,30.657688,-135.0,0.0,461.0,ghoqucicg.Dusa1oyrnkCps8bkh5ttyk,11.839897,0.078753,1,0.0,0.0
2,104.039749,30.660996,-128.0,0.0,219.0,ugBulkik71Fqyg3ouokgCvjbibd-uuuc,12.040412,0.023581,1,0.0,0.0
3,104.039749,30.660996,-128.0,0.0,222.0,jdwnoglcc7Auta0myptcCznnmba3xksm,18.387601,-0.054544,1,0.0,0.0
4,104.039749,30.660996,-128.0,0.0,347.0,ultsjgma89voyi4msypfqvxj8l8@Bmpc,11.237784,0.850043,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
684327,104.127004,30.661507,-127.0,138.0,269.0,naByr9bf76xqubbqpxqfBAqehdc_BxBl,17.632109,-1.187726,1,0.0,0.0
684328,104.127014,30.661517,-127.0,138.0,468.0,inwtw9dfi_Ftvi_owwtlqnw87af4uwsc,19.022417,0.449095,1,0.0,0.0
684329,104.127014,30.661507,-127.0,138.0,343.0,njxusmfa62xxwh9erpmdpnjckla9tqwo,15.914934,0.286259,1,0.0,0.0
684330,104.127034,30.661527,-127.0,138.0,313.0,lapztginj3ytzibrDtifCprddmc-sqwd,17.304050,-0.017318,1,0.0,0.0
